In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glass/glass.csv


In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder

In [3]:
data = pd.read_csv("/kaggle/input/glass/glass.csv",delimiter=",")

In [4]:
data.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [5]:
from sklearn.utils import shuffle
data=shuffle(data)
data

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
67,1.52152,13.05,3.65,0.87,72.32,0.19,9.85,0.00,0.17,1
63,1.52227,14.17,3.81,0.78,71.35,0.00,9.69,0.00,0.00,1
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.00,7
120,1.51844,13.25,3.76,1.32,72.40,0.58,8.42,0.00,0.00,2
114,1.51847,13.10,3.97,1.19,72.44,0.60,8.43,0.00,0.00,2
...,...,...,...,...,...,...,...,...,...,...
109,1.51818,13.72,0.00,0.56,74.45,0.00,10.99,0.00,0.00,2
86,1.51569,13.24,3.49,1.47,73.25,0.38,8.03,0.00,0.00,2
78,1.51613,13.92,3.52,1.25,72.88,0.37,7.94,0.00,0.14,2
105,1.52475,11.45,0.00,1.88,72.19,0.81,13.24,0.00,0.34,2


In [6]:
# split into input (X) and output (Y) variables

X = data.values[0:,0:9].astype(float)
Y = data.values[0:,9]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

In [7]:
X

array([[ 1.52152, 13.05   ,  3.65   , ...,  9.85   ,  0.     ,  0.17   ],
       [ 1.52227, 14.17   ,  3.81   , ...,  9.69   ,  0.     ,  0.     ],
       [ 1.51623, 14.14   ,  0.     , ...,  9.18   ,  1.06   ,  0.     ],
       ...,
       [ 1.51613, 13.92   ,  3.52   , ...,  7.94   ,  0.     ,  0.14   ],
       [ 1.52475, 11.45   ,  0.     , ..., 13.24   ,  0.     ,  0.34   ],
       [ 1.52777, 12.64   ,  0.     , ..., 14.4    ,  0.     ,  0.     ]])

In [8]:
data["Type"].value_counts()

2    76
1    70
7    29
3    17
5    13
6     9
Name: Type, dtype: int64

In [9]:
from sklearn.preprocessing import StandardScaler

Xn = StandardScaler().fit_transform(X)

Xn.shape

(214, 9)

In [10]:
Xn

array([[ 1.04119781, -0.43924559,  0.67091357, ...,  0.62897732,
        -0.35287683,  1.16232646],
       [ 1.28874218,  0.93550477,  0.78209928, ...,  0.51628733,
        -0.35287683, -0.5864509 ],
       [-0.7048151 ,  0.8986811 , -1.86551055, ...,  0.157088  ,
         1.78397794, -0.5864509 ],
       ...,
       [-0.73782101,  0.62864085,  0.58057517, ..., -0.7162594 ,
        -0.35287683,  0.85371869],
       [ 2.10728887, -2.40317467, -1.86551055, ...,  3.0165964 ,
        -0.35287683,  2.91110382],
       [ 3.10406751, -0.94250241, -1.86551055, ...,  3.8335988 ,
        -0.35287683, -0.5864509 ]])

In [11]:
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=0)

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (181, 9)
Number transactions y_train dataset:  (181,)
Number transactions X_test dataset:  (33, 9)
Number transactions y_test dataset:  (33,)


In [12]:


sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))



After OverSampling, the shape of train_X: (420, 9)
After OverSampling, the shape of train_y: (420,) 



In [13]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.3.1


In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(6,activation='softmax')
])

In [32]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(y_train_res)

In [34]:
model.fit(X_train_res,dummy_y , epochs=250)

Epoch 1/250
14/14 [==============================] - 0s 2ms/step - loss: 2.5686 - accuracy: 0.1881
Epoch 2/250
14/14 [==============================] - 0s 2ms/step - loss: 1.9721 - accuracy: 0.1857
Epoch 3/250
14/14 [==============================] - 0s 2ms/step - loss: 1.7417 - accuracy: 0.2905
Epoch 4/250
14/14 [==============================] - 0s 1ms/step - loss: 1.6622 - accuracy: 0.3071
Epoch 5/250
14/14 [==============================] - 0s 1ms/step - loss: 1.6649 - accuracy: 0.3167
Epoch 6/250
14/14 [==============================] - 0s 1ms/step - loss: 1.5645 - accuracy: 0.3905
Epoch 7/250
14/14 [==============================] - 0s 2ms/step - loss: 1.5102 - accuracy: 0.3667
Epoch 8/250
14/14 [==============================] - 0s 2ms/step - loss: 1.5077 - accuracy: 0.4238
Epoch 9/250
14/14 [==============================] - 0s 2ms/step - loss: 1.6728 - accuracy: 0.3310
Epoch 10/250
14/14 [==============================] - 0s 2ms/step - loss: 1.4055 - accuracy: 0.4738
Epoch 11/

In [35]:
dummy_y2 = np_utils.to_categorical(y_test)

In [36]:
test_loss, test_acc = model.evaluate(X_test,  dummy_y2, verbose=2)

print('\nTest accuracy:', test_acc)


2/2 - 0s - loss: 0.8509 - accuracy: 0.7576

Test accuracy: 0.7575757503509521


In [ ]:
y_test